# NLP — Full text search

Piotr Janczyk

In [1]:
from glob import glob
from pathlib import Path
import re
import requests
from tqdm import tqdm
import pandas as pd

%load_ext restmagic

base_url = 'http://localhost:9200'

In [2]:
%%rest_root $base_url
Content-Type: application/json

Requests defaults are set.


### _3. Define an ES analyzer for Polish texts \[...\]_
### _4. Define an ES index for storing the contents of the legislative acts._

I'm creating an index `acts` with single property `content` of type `text`.

The property uses custom analyzer `act_analyzer` which has filters `lowercase`, `graph_synonyms`, and `morfologik_stem`.

In [3]:
%rest -q DELETE acts

<Response [200]>

In [4]:
%%rest PUT acts

{
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "act_analyzer": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "lowercase",
              "graph_synonyms",
              "morfologik_stem"
            ]
          }
        },
        "filter": {
          "graph_synonyms": {
            "type": "synonym_graph",
            "synonyms": [
              "kpk,kodeks postępowania karnego",
              "kpc,kodeks postępowania cywilnego",
              "kk,kodeks karny",
              "kc,kodeks cywilny"
            ]
          }
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "content": {
        "type": "text",
        "analyzer": "act_analyzer"
      }
    }
  }
}

{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "acts"
}

<Response [200]>

Example usage of the custom analyzer:

In [5]:
%%rest GET acts/_analyze

{
    "analyzer": "act_analyzer",
    "text": "KPC stanowi, że"
}

{
  "tokens": [
    {
      "token": "kodeks",
      "start_offset": 0,
      "end_offset": 3,
      "type": "SYNONYM",
      "position": 0
    },
    {
      "token": "kodeks postępowania cywilnego",
      "start_offset": 0,
      "end_offset": 3,
      "type": "<ALPHANUM>",
      "position": 0,
      "positionLength": 3
    },
    {
      "token": "postępowanie",
      "start_offset": 0,
      "end_offset": 3,
      "type": "SYNONYM",
      "position": 1
    },
    {
      "token": "postępować",
      "start_offset": 0,
      "end_offset": 3,
      "type": "SYNONYM",
      "position": 1
    },
    {
      "token": "cywilny",
      "start_offset": 0,
      "end_offset": 3,
      "type": "SYNONYM",
      "position": 2
    },
    {
      "token": "stanowić",
      "start_offset": 4,
      "end_offset": 11,
      "type": "<ALPHANUM>",
      "position": 3
    },
    {
      "token": "stanowy",
      "start_offset": 4,
      "end_offset": 11,
      "type": "<ALPHANUM>",
      "position": 3

<Response [200]>

"KCP" was correctly converted to lowercase, then matched with the synonym "kodeks postępowania cywilnego", and then lematized.

### _5. Load the data to the ES index._

In [6]:
filenames = sorted(glob('repo/ustawy/*.txt'))

for filename in tqdm(filenames):
    id = re.match(r'.*/(\d+_\d+)\.txt', filename).group(1)
    content = Path(filename).read_text()

    requests.put(f'{base_url}/acts/_doc/{id}',
                 json={'content': content})

100%|██████████| 1180/1180 [00:47<00:00, 24.90it/s]


### _6. Determine the number of legislative acts containing the word **ustawa** (in any form)_

In [7]:
%%rest GET acts/_search?filter_path=hits.total.value,hits.hits.highlight.content

{
  "query": {
    "match": {
      "content": "ustawa"
    }
  },
  "highlight": {
    "fields": {
      "content": {
        "fragment_size": 1
      }
    }
  },
  "size": 2
}

{
  "hits": {
    "total": {
      "value": 1179
    },
    "hits": [
      {
        "highlight": {
          "content": [
            "<em>ustawy</em>",
            " \n<em>ustawa</em>",
            "<em>ustawy</em>",
            "<em>ustaw</em>",
            "<em>ustawie</em>"
          ]
        }
      },
      {
        "highlight": {
          "content": [
            "<em>ustawy</em>",
            " \n<em>USTAWA</em>",
            "<em>ustawy</em>",
            "<em>ustaw</em>",
            "<em>ustawie</em>"
          ]
        }
      }
    ]
  }
}

<Response [200]>

Answer:

In [8]:
_.json()['hits']['total']['value']

1179

### _7. Determine the number of legislative acts containing the words **kodeks postępowania cywilnego** in the specified order, but in an any inflection form._

I'm using `match_phrase` to match words in the specific order.

In [9]:
%%rest GET acts/_search?filter_path=hits.total.value,hits.hits.highlight.content

{
  "query": {
    "match_phrase": {
      "content": "kodeks postępowania cywilnego"
    }
  },
  "highlight": {
    "fields": {
      "content": {
        "fragment_size": 1,
        "type": "plain"
      }
    }
  },
  "size": 2
}

{
  "hits": {
    "total": {
      "value": 100
    },
    "hits": [
      {
        "highlight": {
          "content": [
            " – <em>Kodeks postępowania cywilnego</em>",
            " – <em>Kodeks postępowania\ncywilnego</em>",
            ". – <em>Kodeks postępowania cywilnego</em>",
            "\n− <em>Kodeks postępowania cywilnego</em>",
            ". – <em>Kodeks\npostępowania cywilnego</em>"
          ]
        }
      },
      {
        "highlight": {
          "content": [
            " – <em>Kodeks postępowania cywilnego</em>",
            ". – <em>Kodeks postępowania cywilnego</em>"
          ]
        }
      }
    ]
  }
}

<Response [200]>

Answer:

In [10]:
_.json()['hits']['total']['value']

100

### _8. Determine the number of legislative acts containing the words **wchodzi w życie**    (in any form) allowing for up to 2 additional words in the searched phrase._

I'm using `"slop": 2` to allow additional words between the words of the phrase.

In [11]:
%%rest GET acts/_search?filter_path=hits.total.value,hits.hits.highlight.content

{
  "query": {
    "match_phrase": {
      "content": {
        "query": "wchodzi w życie",
        "slop": 2
      }
    }
  },
  "highlight": {
    "fields": {
      "content": {
        "fragment_size": 15,
        "type": "plain"
      }
    }
  },
  "size": 2
}

{
  "hits": {
    "total": {
      "value": 1175
    },
    "hits": [
      {
        "highlight": {
          "content": [
            "\", <em>wchodzi</em> <em>w</em> <em>życie</em> z",
            ", <em>wchodzą</em> <em>w</em> <em>życie</em> z",
            " <em>wchodzą</em> <em>w</em> <em>życie</em> z",
            ", <em>wchodzą</em>\n  <em>w</em> <em>życie</em> nie",
            "\n  województwa, <em>wchodzą</em> <em>w</em> <em>życie</em> z"
          ]
        }
      },
      {
        "highlight": {
          "content": [
            ". Ustawa <em>wchodzi</em> <em>w</em> <em>życie</em> po",
            ", które <em>wchodzą</em> <em>w</em> <em>życie</em> po",
            " <em>wchodzi</em> <em>w</em> <em>życie</em> z",
            ", który <em>wchodzi</em> <em>w</em> <em>życie</em> z",
            " <em>wchodzi</em> <em>w</em> <em>życie</em> z"
          ]
        }
      }
    ]
  }
}

<Response [200]>

Answer:

In [12]:
_.json()['hits']['total']['value']

1175

### _9. Determine the 10 documents that are the most relevant for the phrase **konstytucja**._

In [13]:
%%rest -q GET acts/_search?filter_path=hits.hits._id,hits.hits._score

{
  "size": 10,
  "query": {
    "match": {
      "content": "konstytucja"
    }
  }
}

<Response [200]>

10 most relevant documents:

In [14]:
pd.DataFrame(_.json()['hits']['hits'])

,_id,_score
0,1997_629,6.871580
1,2000_443,6.667763
2,1997_604,6.637804
3,1996_350,6.631998
4,1997_642,6.256372
5,2001_23,6.065163
6,1996_199,5.935479
7,1999_688,5.856316
8,1997_681,5.473405
9,2001_1082,5.473405


### _10. Print the excerpts containing the word **konstytucja** (up to three excerpts per document) from the previous task._

In [15]:
%%rest GET acts/_search?filter_path=hits.hits._id,hits.hits.highlight

{
  "query": {
    "match": {
      "content": "konstytucja"
    }
  },
  "highlight": {
    "fields": {
      "content": {
        "number_of_fragments": 3
      }
    }
  },
  "size": 2
}

{
  "hits": {
    "hits": [
      {
        "_id": "1997_629",
        "highlight": {
          "content": [
            "o zmianie ustawy konstytucyjnej o trybie przygotowania\n           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej",
            "W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i \nuchwalenia <em>Konstytucji</em>",
            "Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz \n                obywateli popierających zgłoszenie"
          ]
        }
      },
      {
        "_id": "2000_443",
        "highlight": {
          "content": [
            "umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89\n     ust. 1 lub art. 90 <em>Konstytucji</em>",
            "międzynarodowej lub załącznika nie\n     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>",
            "co do zasadności wyboru\n  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2\n  <em>K

<Response [200]>